In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def read_corpus(fname):
    for line in open(fname):
        tokens = line.split()
        words = tokens[1:]
        yield ' '.join(words)
def read_labels(fname):
    to_return = []
    for line in open(fname):
        tokens = line.split()
        label = 1 if tokens[0] == 'ham' else 0
        to_return.append(label)
    return to_return
        
vectorizer = TfidfVectorizer(max_features=300, max_df=0.1, min_df=0.01, sublinear_tf=True)
all_docs = read_corpus('SMSSpamCollection')
tfidf_matrix = vectorizer.fit_transform(all_docs)

import pandas as pd
feature_names = vectorizer.get_feature_names()
dense = tfidf_matrix.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)
df

In [ ]:
from sklearn import linear_model
classifier = linear_model.LogisticRegression()
all_regressors = tfidf_matrix.todense().tolist()
train_labels = read_labels('SMSSpamCollectionTrain')
test_labels = read_labels('SMSSpamCollectionTest')
classifier.fit(all_regressors[:750], train_labels)
test_predictions = classifier.predict(all_regressors[750:])
len_predictions = len(test_predictions)
print(len(test_predictions))
print(len(test_labels))
corrects = sum(test_predictions == test_labels)
errors = len_predictions - corrects
error_rate = float(errors)/len_predictions
print("Error: {0}; Error Count: {1}; Test Count {2}; \n".format(error_rate, errors, len_predictions))

In [ ]:
all_docs = read_corpus("SMSSpamCollection")
all_docs = [doc for doc in all_docs]
###
#1
#all_docs.append("GMAIL LinkedIn Daniel, people found you in search this week Daniel, you're getting noticed. See your search appearances in the Linked...")

#1
#all_docs.append("MESSENGER janga sent you a message")

#1
#all_docs.append("PHOTOS Emika New photo from Ming Zou")

#1
#all_docs.append("SNAPCHAT from Alan Fang")

#1
#all_docs.append("MOMENT Insights Your daily insight report is ready. 3H 1M screen time yesterday 30M from the day before.")

#1
#all_docs.append("UBER Thanks for riding with Daniel! Please rate your trip, and you can also add a tip.")

#1
#all_docs.append("Jason wants you play Candy Crush Saga")

#1
all_docs.append("hello Come to Chipotle Mexican Grill There are Free chips and guacamole")
###
train_labels = read_labels('SMSSpamCollectionTrain')
all_regressors = vectorizer.fit_transform(all_docs).todense().tolist()
classifier = linear_model.LogisticRegression()
classifier.fit(all_regressors[:750], train_labels)
test_prediction = classifier.predict(all_regressors[-1])
print(test_prediction) # 1 if ham

###  Now begins an attempt to use pretrained word2vec from PaddlePaddle by concatenating dictionary words together from word2vec and running scikit-learn logistic regression

In [ ]:
from scipy import spatial
import numpy

def load_dict_and_embedding():
    word_dict = dict()
    with open("word_dict", "r") as f:
        for line in f:
            key, value = line.strip().split(" ")
            word_dict[key] = int(value)

    embeddings = numpy.loadtxt("embedding_table", delimiter=",")
    return word_dict, embeddings

# load word dict and embedding table
word_dict, embedding_table = load_dict_and_embedding()

my_embedding = [0]*32
for word in all_docs[-1].split(' '):
    if (word.lower() in word_dict):
        my_embedding += embedding_table[word_dict[word.lower()]]
    else:
        print("{} not in word table".format(word.lower()))
print(my_embedding)
#print(spatial.distance.cosine(embedding_table[word_dict['car']], embedding_table[word_dict['world']]))
#print(spatial.distance.cosine(embedding_table[word_dict['say']], embedding_table[word_dict['talking']]))

## Start from here

In [ ]:
import string
def read_corpus(fname):
    for line in open(fname):
        tokens = line.split()
        words = tokens[1:]
        yield ' '.join(words)
def read_labels(fname):
    to_return = []
    for line in open(fname):
        tokens = line.split()
        label = 1 if tokens[0] == 'ham' else 0
        to_return.append(label)
    return to_return
from scipy import spatial
import numpy

from string import punctuation
def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

def load_dict_and_embedding():
    word_dict = dict()
    with open("word_dict", "r") as f:
        for line in f:
            key, value = line.strip().split(" ")
            word_dict[key] = int(value)

    embeddings = numpy.loadtxt("embedding_table", delimiter=",")
    return word_dict, embeddings

# load word dict and embedding table
word_dict, embedding_table = load_dict_and_embedding()

In [ ]:
all_docs = read_corpus("SMSSpamCollection")
all_regressors = []
my_embedding = [0]*32
for doc in all_docs:
    #doc = strip_punctuation(doc)
    i = 0
    for word in doc.split(' '):
        if (word.lower() in word_dict):
            my_embedding += embedding_table[word_dict[word.lower()]]
            i+=1
    if i == 0:
        i = 1
    all_regressors.append([x/i for x in my_embedding])
        #else:
            #print("{} not in word table".format(word.lower()))
print(all_regressors)

In [ ]:
from sklearn import linear_model
classifier = linear_model.LogisticRegression()
train_labels = read_labels('SMSSpamCollectionTrain')
test_labels = read_labels('SMSSpamCollectionTest')
classifier.fit(all_regressors[:750], train_labels)
test_predictions = classifier.predict(all_regressors[750:750+len(test_labels)])
len_predictions = len(test_predictions)
print(len(test_predictions))
print(len(test_labels))
corrects = sum(test_predictions == test_labels)
errors = len_predictions - corrects
error_rate = float(errors)/len_predictions
print("Error: {0}; Error Count: {1}; Test Count {2}; \n".format(error_rate, errors, len_predictions))

In [ ]:
def paddle_predict(inpstring):
    my_embedding = [0]*32
    for word in inpstring.split(' '):
        if word.lower() in word_dict:
            my_embedding+=embedding_table[word_dict[word.lower()]]
    test_prediction = classifier.predict(my_embedding)
    return test_prediction # 1 if ham

In [ ]:
paddle_predict("free free free free free free")

#### wow it sure seems like using word2vec outputs 1 alot, probably because spam words are OOV. Let's do ensemble

In [1]:
import string
def read_corpus(fname):
    for line in open(fname):
        tokens = line.split()
        words = tokens[1:]
        yield ' '.join(words)
def read_labels(fname):
    to_return = []
    for line in open(fname):
        tokens = line.split()
        label = 1 if tokens[0] == 'ham' else 0
        to_return.append(label)
    return to_return
from scipy import spatial
import numpy

from string import punctuation
def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

def load_dict_and_embedding():
    word_dict = dict()
    with open("word_dict", "r") as f:
        for line in f:
            key, value = line.strip().split(" ")
            word_dict[key] = int(value)

    embeddings = numpy.loadtxt("embedding_table", delimiter=",")
    return word_dict, embeddings

# load word dict and embedding table
word_dict, embedding_table = load_dict_and_embedding()

In [2]:
all_docs = read_corpus("SMSSpamCollection")
all_regressors = []
my_embedding = [0]*32
for doc in all_docs:
    #doc = strip_punctuation(doc)
    i = 0
    for word in doc.split(' '):
        if (word.lower() in word_dict):
            my_embedding += embedding_table[word_dict[word.lower()]]
            i+=1
    if i == 0:
        i = 1
    all_regressors.append([x/i for x in my_embedding])
        #else:
            #print("{} not in word table".format(word.lower()))
print(all_regressors)

[[-0.019518812807897728, -0.0045128345696462523, -0.00087049148148960539, -0.010788504448201921, -0.04490387522512012, 0.038248642037312187, 0.018970362635122404, -0.024133459147479799, 0.0085748609983258769, 0.09626023768861261, 0.030424413892130058, -0.0074630144776569474, 0.028341993068655331, -0.011745923198759556, 0.0031235563930951888, 0.031715628678082593, -0.023267478035348985, -0.053392157320761018, 0.00021958684859176478, 0.039453940362566046, 0.0041721588931977749, 0.014174069976434112, -0.0038929140816132226, 0.047316448659532599, -0.050346279309855566, -0.0030905856854385799, -0.0019100243225693703, 0.011021266277465556, -0.012339053882492913, 0.008672918710443709, -0.0025985563794771829, 0.03574623167514801], [-0.17566931527107954, -0.040615511126816273, -0.0078344233334064484, -0.097096540033817291, -0.40413487702608109, 0.34423777833580971, 0.17073326371610165, -0.21720113232731819, 0.077173748984932899, 0.86634213919751346, 0.27381972502917051, -0.067167130298912525, 0

In [3]:
from sklearn import linear_model
classifier = linear_model.LogisticRegression()
train_labels = read_labels('SMSSpamCollectionTrain')
test_labels = read_labels('SMSSpamCollectionTest')
classifier.fit(all_regressors[:750], train_labels)
test_predictions = classifier.predict(all_regressors[750:])
len_predictions = len(test_predictions)
print(len(test_predictions))
print(len(test_labels))
corrects = sum(test_predictions == test_labels)
errors = len_predictions - corrects
error_rate = float(errors)/len_predictions
print("Error: {0}; Error Count: {1}; Test Count {2}; \n".format(error_rate, errors, len_predictions))

241
241
Error: 0.17842323651452283; Error Count: 43; Test Count 241; 



In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=300, max_df=0.1, min_df=0.01, sublinear_tf=True)
all_docs = read_corpus('SMSSpamCollection')
tfidf_matrix = vectorizer.fit_transform(all_docs)

import pandas as pd
feature_names = vectorizer.get_feature_names()
dense = tfidf_matrix.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)
df

classifier2 = linear_model.LogisticRegression()
all_regressors2 = tfidf_matrix.todense().tolist()
train_labels = read_labels('SMSSpamCollectionTrain')
test_labels = read_labels('SMSSpamCollectionTest')
classifier2.fit(all_regressors2[:750], train_labels)
test_predictions = classifier2.predict(all_regressors2[750:])
len_predictions = len(test_predictions)
print(len(test_predictions))
print(len(test_labels))
corrects = sum(test_predictions == test_labels)
errors = len_predictions - corrects
error_rate = float(errors)/len_predictions
print("Error: {0}; Error Count: {1}; Test Count {2}; \n".format(error_rate, errors, len_predictions))

all_docs = read_corpus("SMSSpamCollection")
all_docs = [doc for doc in all_docs]
classifier2 = linear_model.LogisticRegression()
classifier2.fit(all_regressors2[:750], train_labels)

241
241
Error: 0.08298755186721991; Error Count: 20; Test Count 241; 



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [5]:
import numpy as np
def paddle_predict(inpstring):
    my_embedding = [0]*32
    for word in inpstring.split(' '):
        if word.lower() in word_dict:
            my_embedding+=embedding_table[word_dict[word.lower()]]
    test_prediction = classifier.predict(my_embedding)
    all_docs.append(inpstring)
    train_labels = read_labels('SMSSpamCollectionTrain')
    all_regressors2 = vectorizer.fit_transform(all_docs).todense().tolist()
    test_prediction2 = classifier2.predict(all_regressors2[-1])
    print("model1: {}, model2: {}".format(test_prediction2, test_prediction))
    return np.rint(0.3*test_prediction+0.7*test_prediction2) # 1 if ham

In [12]:
paddle_predict("i love you babe, call 555 for a good time(for free free free free free free call me call me call me 38382737237 433842734823 phone free call me call me you won won won won  free)")

model1: [0], model2: [1]


/Users/daniel/miniconda3/envs/py3/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/daniel/miniconda3/envs/py3/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([ 0.])